In [1]:
import glob
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import re
import os
import time
from urllib.parse import urlparse
import random
from multiprocessing.pool import ThreadPool
from IPython.display import clear_output

all_txt_files = glob.glob(os.getcwd()+'\\prod_urls\\*.txt')
driver = None
dumb_urls = ['https://www.thepythoncode.com/','https://www.geeksforgeeks.org/','https://pynative.com/','http://facebook.com/','https://www.amazon.in/','https://www.tutorialspoint.com/','https://docs.python.org/','https://chromedriver.chromium.org/','https://github.com/','https://twitter.com/']

home_dir = os.getcwd()
results_dir = home_dir + '\\results'
images_dir = results_dir + '\\images'

all_dataframes = dict()

In [2]:
prod_url_txt_files = [txt_file for txt_file in glob.glob(home_dir+'\\prod_urls\\Arts, Language and Linguistic Books.txt')]

genres_and_links = dict()
for filename in prod_url_txt_files:
    with open(filename,'r') as file:
        relative_filename = filename.split('\\')[-1]
        genres_and_links[relative_filename.replace('.txt','')] = file.readlines()
prod_url_txt_files

['E:\\projects-ds-ws\\webscraping-product-details-flipkart\\prod_urls\\Arts, Language and Linguistic Books.txt']

In [3]:
for kk,vv in genres_and_links.items():
    print(len(vv),'-',kk)

1000 - Arts, Language and Linguistic Books


In [4]:
def replace_multispaces_with_single_space(text):
    text = re.sub('\\s+',' ',text)
    return text

In [5]:
def save_image(soup,main_counter,genre,link,driver,existing_images_list):
    image_urls = set()
    image_url = None
    i_names = list()

    i_file_name = results_dir + '\\images\\' + genre + '\\' +str(main_counter)+'.jpeg'
    if i_file_name not in existing_images_list:
        #driver.get(dumb_urls[random.randint(0,len(dumb_urls)-1)])
        #driver.get(link)
        #page_source = driver.page_source
        #soup = BeautifulSoup(page_source,'html.parser')
        image_div_ele = soup.find('div',{'class':'_1ov7-N'})
        if image_div_ele:
            img = image_div_ele.find('img')
            if img:
                if img.has_attr('src'):
                    image_url = img.get('src')
        if image_url is not None:
            image_url = re.sub('\\w+?q=\\d+','',image_url)
            image_url = re.sub('https://rukminim1.flixcart.com/image/\\d+/\\d+/','https://rukminim1.flixcart.com/image/512/512/',image_url)
            url_parse_results = urlparse(image_url)
            if all([url_parse_results.scheme, url_parse_results.netloc, url_parse_results.path]):
                image_urls.add(image_url)
    else:
        i_name = i_file_name.split('\\')[-1]
        i_names.append(i_name)
        return i_names;

    if len(image_urls)==0:
        image_url = 'https://img1a.flixcart.com/www/linchpin/fk-cp-zion/img/flipkart-plus_4ee2f9.png'
        image_urls.add(image_url)
            
    for i_url in image_urls:
        if i_file_name not in existing_images_list:
            requests.get(dumb_urls[random.randint(0,len(dumb_urls)-1)])
            i_response = requests.get(i_url)
            with open(i_file_name, 'wb') as file:
                file.write(i_response.content)
        i_name = i_file_name.split('\\')[-1]
        i_names.append(i_name)
        break;
    return i_names;

In [6]:
def collect_data(soup,link,dumb_urls):
    title = 'NA'
    price = 'NA'
    publisher = 'NA'
    no_of_pages = 'NA'
    isbn = 'NA'
    language = 'NA'
    rating = 'NA'
    author = 'NA'
    summary = 'NA'
    
    title_ele = soup.find('span',{'class':'_35KyD6'})
    title = title_ele.get_text()
    title = replace_multispaces_with_single_space(title)
        
    price_ele = soup.find('div',{'class':'_1vC4OE _3qQ9m1'})
    if price_ele:
        price = price_ele.get_text()
        price = re.sub('₹','',price)
            
    spec_tbody_elements = soup.find_all('tr',{'class':'_3_6Uyw row'})
    for tbody in spec_tbody_elements:
        tr_elements = tbody.find_all('td')
        if len(tr_elements)==2:
            if tr_elements[0] and tr_elements[0].text=='Imprint':
                publisher = tr_elements[1].get_text()
    
    highlights_li_elements = soup.find_all('li',{'class':'_2-riNZ'})
    for li in highlights_li_elements:
        if li.get_text() and li.get_text().startswith('Pages'):
            no_of_pages = li.get_text().split(': ')[1]
        if li.get_text() and li.get_text().startswith('ISBN'):
            isbn = li.get_text().split(': ')[1]
        if li.get_text() and li.get_text().startswith('Language'):
            language = li.get_text().split(': ')[1]
        
    rating_parent_element = soup.find('div',{'class':'_3ors59'})
    if rating_parent_element:
        rating_element = rating_parent_element.find('div',{'class':'hGSR34'})
        if rating_element:
            rating = rating_element.get_text()
        
    book_desc_list = list()
    author_description_elements = soup.find_all('div',{'class':'_1y9a40'})
    for element in author_description_elements:
        sub_ele = element.find('div',{'class':'_1oCqc9'})
        if sub_ele and sub_ele.get_text()=='Author':
            author = element.find('div',{'class':'_3cpW1u'}).get_text().strip()
        if sub_ele and sub_ele.get_text()=='Description':
            sub_sub_ele = element.find('div',{'class':'_3cpW1u'})
            if sub_sub_ele:
                summary = sub_sub_ele.get_text()
                summary = replace_multispaces_with_single_space(summary)

    return (title,price,publisher,no_of_pages,isbn,language,rating,author,summary)

In [ ]:
driver = None

def scrap_data(links,driver,genre):

    titles = list()
    prices = list()
    publishers = list()
    list_of_no_of_pages = list()
    isbns = list()
    genres = list()
    ratings = list()
    authors = list()
    summaries = list()
    image_names = list()
    languages= list()
    
    main_counter = 1
    for link in links:
        if driver is None:
            #options = webdriver.ChromeOptions()
            #options.add_argument("--start-maximized")
            #driver = webdriver.Chrome(executable_path="chromedriver.exe",options=options)
            driver = webdriver.Firefox(executable_path='geckodriver.exe')
        
        driver.get(dumb_urls[random.randint(0,len(dumb_urls)-1)])
        driver.get(link)
        page_source = driver.page_source
        soup = BeautifulSoup(page_source,'html.parser')
        
        #async_image_results = pool1.apply_async(save_image, (soup,main_counter,genre,link,driver,all_image_list))
        i_names = save_image(soup,main_counter,genre,link,driver,all_image_list)
        #async_other_results = pool2.apply_async(collect_data, (soup,link,dumb_urls))
        title,price,publisher,no_of_pages,isbn,language,rating,author,summary = collect_data(soup,link,dumb_urls)

        #i_names = async_image_results.get()
        #book_title,price,publisher,no_of_pages,isbn,language,rating,author,book_summary = async_other_results.get()
                
        titles.append(title)
        prices.append(price)
        publishers.append(publisher)
        list_of_no_of_pages.append(no_of_pages)
        isbns.append(isbn)
        genres.append(genre.replace(' Books',''))
        ratings.append(rating)
        authors.append(author)
        summaries.append(summary)
        image_names.append(', '.join(i_names))
        languages.append(language)
        
        print(main_counter)
        main_counter += 1

        if main_counter%100==0:
        #if False:
            clear_output()
            print(genre)
            temp_dict = dict()
            temp_dict['title'] = titles
            temp_dict['price'] = prices
            temp_dict['publishers'] = publishers
            temp_dict['number_of_pages'] = list_of_no_of_pages
            temp_dict['isbn_10'] = isbns
            temp_dict['isbn_13'] = isbns
            temp_dict['genres'] = genres
            temp_dict['rating'] = ratings
            temp_dict['authors'] = authors
            temp_dict['description'] = summaries
            temp_dict['image'] = image_names
            temp_dict['thumb'] = image_names
            temp_dict['languages'] = languages
            temp_df = pd.DataFrame(temp_dict)
            temp_df = temp_df[['isbn_10','isbn_13','image','thumb','title','authors','languages','genres','number_of_pages','description','publishers','rating','price']]
            export_file_name = results_dir + '\\' + genre+'_'+str(main_counter) + '.csv'
            temp_df.to_csv(export_file_name,index=False)

    return titles,prices,publishers,list_of_no_of_pages,isbns,genres,ratings,authors,summaries,image_names,languages

for genre, links in genres_and_links.items():
    
    results_dict = dict()
    titles,prices,publishers,list_of_no_of_pages,isbns,genres,ratings,authors,summaries,image_names,languages = None,None,None,None,None,None,None,None,None,None,None

    #create images folder for each genre
    genre_images_dir = results_dir + '\\images\\' + genre
    if not os.path.exists(genre_images_dir):
        os.makedirs(genre_images_dir)
    all_image_list = glob.glob(genre_images_dir+'\\*.jpeg')
    
    pool = ThreadPool(processes=7)
    #pool2 = ThreadPool(processes=2)
    
    async_resp = pool.apply_async(scrap_data,(links,driver,genre))

    (titles,prices,publishers,list_of_no_of_pages,isbns,genres,ratings,authors,summaries,image_names,languages) = async_resp.get()

    results_dict['title'] = titles
    results_dict['price'] = prices
    results_dict['publishers'] = publishers
    results_dict['number_of_pages'] = list_of_no_of_pages
    results_dict['isbn_10'] = isbns
    results_dict['isbn_13'] = isbns
    results_dict['genres'] = genres
    results_dict['rating'] = ratings
    results_dict['authors'] = authors
    results_dict['description'] = summaries
    results_dict['image'] = image_names
    results_dict['thumb'] = image_names
    results_dict['languages'] = languages
    
    df = pd.DataFrame(results_dict)
    df = df[['isbn_10','isbn_13','image','thumb','title','authors','languages','genres','number_of_pages','description','publishers','rating','price']]
    export_file_name = results_dir + '\\' + genre + '.csv'
    df.to_csv(export_file_name,index=False)
    ultimate_dataframes[genre] = df
    print('yahooooooo!!! Completed for','\"',genre,'\"')

Arts, Language and Linguistic Books
300
301
302
303
304


In [ ]:
if driver is not None:
    driver.quit()

In [ ]:
print('Everything completed. Thank god.')